<a href="https://colab.research.google.com/github/mrohani/Google-Trends-Suggestions/blob/main/Google_Trends_Suggestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from time import sleep
from tqdm.notebook import tqdm
from random import randint
!pip install pytrends
from pytrends.request import TrendReq
from datetime import date

In [2]:
# Initiate PyTrends
pytrends = TrendReq()

# Define paremeters to use in our script

## Google Trends ##

# Trends country, Country to use in Google Trends
# Not all the countries are supported, waiting for my country (Jordan) to be supported 
# https://support.google.com/trends/answer/4355212?hl=en
trending_country = "united_states"

## Google Suggestions ##

# Suggestions language, currenlty this script supports only Arabic (ar) and English (en) languages
# fa => coming soon
language = "en"

# Letters to use in Google Suggestions, currently, you can use only Arabic and English letters
# Feel free to add your language
letters = {'ar':['أ','ب','ت','ث','ج','ح','خ','د','ذ','ر','ز','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','ه','و','ي'],
          'en':['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
           'fa':['الف','ب','پ','ت','ث','ج','چ','ح','خ','د','ذ','ر','ز','ژ','س','ش','ص','ط','ع','غ','ف','ق','ک','گ','ل','م','ن','و','ه','ی']
          }

# Suggestions Country code, you can use any country code, like jo for Jordan.
# sa for Saudi Arabia, full list https://bit.ly/3bMiAHz
country = "united_States"


# User Agent string to use in our requests, Google Chrome
ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

# Delay in seconds between each request to avoid getting blocked by Google
delay = 1

# Date to use in our final CSV file
output_date = date.today().strftime("%b-%d-%Y").lower()

# Output file name
output_file_name = f"google-trends-suggestions_{output_date}_{trending_country}_{randint(1,9999)}.csv"

# Empty lists to use later
letters_list = []
original_keywords_list = []
word_count_list = []
suggested_list = []
country_list = []
trend_country = []

In [3]:
def get_trends(trend_country):
    return pytrends.trending_searches(pn=trend_country)

In [4]:
def generate_suggestions(trending_searches,suggestions_language,output_file_name,trending_country):
    i = 0
    for trend in tqdm(trending_searches[0]):
        # Replace the spaces from each keyword with "+"
        trend = trend.replace(" ","+")
        for letter in letters[suggestions_language]:
            # Generate the URL for Google Suggest Query
            url = f"http://suggestqueries.google.com/complete/search?output=firefox&q={trend}+{letter}&hl={country}&gl={language}"
            suggest_page = requests.get(url,headers = {"User-Agent":ua})
            # Check the response code
            if suggest_page.status_code == 200:
                suggest_list = suggest_page.json()
                for suggest in suggest_list[1]:
                    letters_list.append(letter)
                    original_keywords_list.append(trend.replace("+"," "))
                    word_count_list.append(len(suggest.split(" ")))
                    trend_country.append(trending_country.capitalize().replace("_"," "))
                    suggested_list.append(suggest)
                    country_list.append(country)
            else:
                # Blocked from Google, try to increase the delay between the requests
                print(suggest_page.status_code)
            sleep(delay)
    
        # Dictionary for the data frame
        suggest_dataframe_list = {"Trend":original_keywords_list,
                             "Suggested Keywords":suggested_list,
                             "Word Count":word_count_list,
                            "Trend Country":trend_country,
                             "Letter":letters_list,
                             "Suggestion Country":country_list
        }
        
        # Generate Dataframe for each Trend keyword to avoid loosing any results
        # incase something wrong happend
        generate_df(suggest_dataframe_list,output_file_name)
        
        # Reset the dictionary values
        suggest_dataframe_list = {}

In [5]:
def generate_df(suggest_dataframe_list,output_file_name):
    # Convert the output to Pandas Dataframe
    suggest_dataframe = pd.DataFrame(suggest_dataframe_list,columns=["Trend","Suggested Keywords","Word Count","Trend Country","Letter","Suggestion Country"])

    # Save the file in a CSV format
    suggest_dataframe.to_csv(output_file_name) 

In [6]:
# Get trends for a certain country
trending_searches = get_trends(trending_country)

# Generate Suggestions from the trends
generate_suggestions(trending_searches,language,output_file_name,trending_country)

print(f"Done, check file {output_file_name}")

# Display sample of the results
display(pd.read_csv(output_file_name, encoding = "utf-8"))


Done, check file google-trends-suggestions_may-30-2021_united_states_4168.csv


,Unnamed: 0,Trend,Suggested Keywords,Word Count,Trend Country,Letter,Suggestion Country
0,0,Mark Eaton,mark eaton accident,3,United states,a,united_States
1,1,Mark Eaton,mark eaton age,3,United states,a,united_States
2,2,Mark Eaton,mark eaton and karl malone,5,United states,a,united_States
3,3,Mark Eaton,mark eaton all star,4,United states,a,united_States
4,4,Mark Eaton,mark eaton and wife,4,United states,a,united_States
...,...,...,...,...,...,...,...
4922,4922,Matthew Perry,matthew perry zanesville ohio,4,United states,z,united_States
4923,4923,Matthew Perry,matthew perry zero dark thirty,5,United states,z,united_States
4924,4924,Matthew Perry,matthew perry zomrel,3,United states,z,united_States
4925,4925,Matthew Perry,matthew perry son,3,United states,z,united_States
